# Multistage Stochastic Transportation Problem

In [ ]:
using JuMP, SDDP, HiGHS, Gurobi, Statistics, Test

## Toy Instance: Single (Entry-Exit) Lane and Two Sources

In [ ]:
# In Julia this struct is immutable
struct HyperParamsSingle
    tau             ::Int64
    nSources        ::Int64
    entry_stock_0   ::Int64
    exit_stock_0    ::Int64
    exit_short_0    ::Int64
    entry_capacity  ::Int64
    exit_capacity   ::Int64
    flow_support    ::Vector{Int64}
    entry_store_coef::Float64
    exit_store_coef ::Float64
    exit_short_coef ::Float64
    transport_coef  ::Vector{Float64}
end

# Outer constructor with arguments
function HyperParamsSingle(; 
                     tau             ::Int64 = 4,
                     nSources        ::Int64 = 2,
                     entry_stock_0   ::Int64 = 5,
                     exit_stock_0    ::Int64 = 0,
                     exit_short_0    ::Int64 = 0,
                     entry_capacity  ::Int64 = 50,
                     exit_capacity   ::Int64 = 50,
                     flow_support    ::Vector{Int64} = collect(1:10),
                     entry_store_coef::Float64 = 2.0,
                     exit_store_coef ::Float64 = 1.0,
                     exit_short_coef ::Float64 = 3.0,
                     transport_coef  ::Vector{Float64} = [1.1, 0.7])
    HyperParamsSingle(
        tau,
        nSources,
        entry_stock_0,
        exit_stock_0,
        exit_short_0,
        entry_capacity,
        exit_capacity,
        flow_support,
        entry_store_coef,
        exit_store_coef,
        exit_short_coef,
        transport_coef)
end

In [ ]:
function transportation_t(sp::Model, stage::Int64; config::HyperParamsSingle)
    ## State variables
    @variables(sp, begin
        0 <= entry, (SDDP.State, initial_value = config.entry_stock_0)
        0 <= exitp, (SDDP.State, initial_value = config.exit_stock_0)
        0 <= exitm, (SDDP.State, initial_value = config.exit_short_0)
        0 <= move[k = 1:config.nSources]
        flow[i = 1:2]
    end)
    ## Decision variables
    ## Constraints
    # Carrier capacity
    @constraint(sp, sum(move) <= entry.in + flow[1])
    # Storage limit
    @constraint(sp, entry.in + flow[1]   <= config.entry_capacity)
    @constraint(sp, exitp.in + sum(move) <= config.exit_capacity)
    # Transition
    @constraint(sp, entry.out == entry.in + flow[1] - sum(move))
    @constraint(sp, exitp.out - exitm.out == exitp.in - exitm.in + sum(move) - flow[2])
    # Uncertain variables
    # Ξ = vec(collect(Base.product(config.flow_support, config.flow_support)))
    Ξ = vec(collect(Base.product(ntuple(_ -> config.flow_support, 2)...)))
    SDDP.parameterize(sp, Ξ) do ξ
        JuMP.fix.(flow, ξ)
    end
    ## Objective function
    @stageobjective(sp, 
        config.entry_store_coef * entry.in + 
        config.exit_store_coef  * exitp.in + 
        config.exit_short_coef  * exitm.in + 
        sum(config.transport_coef[k] * move[k] for k in 1:config.nSources)
    )
end

In [ ]:
# Hyperparameter configuration
config = HyperParamsSingle() # Default

In [ ]:
model = SDDP.PolicyGraph(
    (sp, stage) -> transportation_t(sp, stage; config = config),
    SDDP.LinearGraph(config.tau);
    sense       = :Min,
    lower_bound = 0.0,
    optimizer   = HiGHS.Optimizer,
)

In [ ]:
SDDP.train(model;
    iteration_limit = 100,
    cut_type        = SDDP.MULTI_CUT,
)

## Multi-Dimensional Instance

In [1]:
using JuMP, SDDP, HiGHS, Gurobi, Statistics, Test, DataStructures

In [2]:
# In Julia this struct is immutable
struct HyperParams
    tau             ::Int64
    nOrigins        ::Int64
    nDestinations   ::Int64
    nLanes          ::Int64
    nCarriers       ::Int64
    Bids            ::Vector{Vector{Int64}}
    Winners         ::OrderedDict{Int64, Vector{Int64}}
    CarrierIdx      ::Vector{Vector{Int64}}
    from_           ::Vector{Vector{Int64}}
    to_             ::Vector{Vector{Int64}}
    entry_stock_0   ::Vector{Int64}
    exit_stock_0    ::Vector{Int64}
    exit_short_0    ::Vector{Int64}
    entry_capacity  ::Vector{Int64}
    exit_capacity   ::Vector{Int64}
    flow_support    ::Vector{NTuple}
    entry_store_coef::Vector{Float64}
    exit_store_coef ::Vector{Float64}
    exit_short_coef ::Vector{Float64}
    transport_coef  ::Vector{Float64}
    carrier_capacity::Vector{Int64}
end

# Outer constructor with arguments
function HyperParams(; 
                     tau             ::Int64 = 4,
                     nOrigins        ::Int64 = 2,
                     nDestinations   ::Int64 = 2,
                     nCarriers       ::Int64 = 3,
                     Bids            ::Vector{Vector{Int64}} = [[1, 2, 3, 4],
                                                                [1, 3],
                                                                [2, 4],
                                                                [1, 2],
                                                                [3, 4]],
                     Winners         ::OrderedDict{Int64, Vector{Int64}} = OrderedDict(
                                                                             1 => [2, 3], 
                                                                             2 => [4, 5], 
                                                                             3 => [1]),
                     entry_stock_0   ::Vector{Int64} = [5, 5],
                     exit_stock_0    ::Vector{Int64} = [0, 0],
                     exit_short_0    ::Vector{Int64} = [0, 0],
                     entry_capacity  ::Vector{Int64} = [50, 50],
                     exit_capacity   ::Vector{Int64} = [50, 50],
                     flow_support    ::Vector{Int64} = [0, 5, 10],
                     entry_store_coef::Vector{Float64} = [2.0, 2.0],
                     exit_store_coef ::Vector{Float64} = [1.0, 1.0],
                     exit_short_coef ::Vector{Float64} = [3.0, 3.0],
                     transport_coef  ::Vector{Float64} = [ 2.0, 1.7, 2.1, 3.3,   # Carrier 1
                                                           1.7, 1.9, 2.6, 3.7,   # Carrier 2
                                                           2.4, 1.2, 2.9, 3.1 ], # Carrier 3
                     carrier_capacity::Vector{Int64} = [ 8,  5,  7, 20, 
                                                         7,  5,  8, 20, 
                                                        12,  8,  8, 11])

    # nLanes = nOrigins * nDestinations

    from_ = Dict{Int64, Vector{Int64}}()
    for i in 1:nOrigins
        from_[i] = (collect(1:nDestinations) .- 1) .* nOrigins .+ i
    end

    to_ = Dict{Int64, Vector{Int64}}()
    for j in 1:nDestinations
        to_[j] = (j .- 1) .* nOrigins .+ collect(1:nOrigins)
    end

    ordx = vcat(collect(values(Winners))...)
    Ldx  = vcat(Bids[ordx]...)

    nLc = [0; cumsum([ length(vcat(Bids[Winners[cdx]]...)) for cdx in 1:nCarriers ])]

    HyperParams(
        tau,
        nOrigins,
        nDestinations,
        nLc[end], # nLanes
        nCarriers,
        Bids,
        Winners,
        [ collect(nLc[i] + 1 : nLc[i + 1]) for i in 1:length(nLc) - 1 ], # CarrierIdx
        [ findall(lane -> lane in from_[i], Ldx) for i in 1:nOrigins ],
        [ findall(lane -> lane in to_[j], Ldx) for j in 1:nDestinations ],
        entry_stock_0,
        exit_stock_0,
        exit_short_0,
        entry_capacity,
        exit_capacity,
        vec(collect(Base.product(ntuple(_ -> flow_support, nOrigins + nDestinations)...))),
        entry_store_coef,
        exit_store_coef,
        exit_short_coef,
        transport_coef,
        carrier_capacity)
end

HyperParams

In [9]:
function transportation_t(sp::Model, stage::Int64; config::HyperParams)
    ## State variables
    @variables(sp, begin
        0 <= entry[i = 1:config.nOrigins],      (SDDP.State, initial_value = config.entry_stock_0[i])
        0 <= exitp[j = 1:config.nDestinations], (SDDP.State, initial_value = config.exit_stock_0[j])
        0 <= exitm[j = 1:config.nDestinations], (SDDP.State, initial_value = config.exit_short_0[j])
        0 <= move[k  = 1:config.nLanes]
        inflow[ i = 1:config.nOrigins]
        outflow[j = 1:config.nDestinations]
    end)
    ## Decision variables
    ## Constraints
    # Carrier capacity
    @constraint(sp, [k = 1:config.nCarriers], sum(move[config.CarrierIdx[k]]) <= config.carrier_capacity[(k - 1) * config.tau + stage])
    # Entry volume
    @constraint(sp, [i = 1:config.nOrigins], sum(move[config.from_[i]]) <= entry[i].in + inflow[i])
    # Storage limit
    # @constraint(sp, [i = 1:config.nOrigins],      entry[i].in + inflow[i] <= config.entry_capacity[i])
    @constraint(sp, [j = 1:config.nDestinations], exitp[j].in + sum(move[config.to_[j]]) <= config.exit_capacity[j])
    # Transition
    @constraint(sp, [i = 1:config.nOrigins],      entry[i].out == entry[i].in + inflow[i] - sum(move[config.from_[i]]))
    @constraint(sp, [j = 1:config.nDestinations], exitp[j].out - exitm[j].out == exitp[j].in - exitm[j].in + sum(move[config.to_[j]]) - outflow[j])
    # Uncertain variables
    Ξ = config.flow_support
    SDDP.parameterize(sp, Ξ) do ξ
        JuMP.fix.(inflow,  ξ[1:config.nOrigins])
        JuMP.fix.(outflow, ξ[config.nOrigins .+ (1:config.nDestinations)])
    end
    ## Objective function
    @stageobjective(sp, 
        sum(config.entry_store_coef[i] * entry[i].in for i in 1:config.nOrigins)      + 
        sum(config.exit_store_coef[j]  * exitp[j].in for j in 1:config.nDestinations) + 
        sum(config.exit_short_coef[j]  * exitm[j].in for j in 1:config.nDestinations) + 
        # sum(config.transport_coef[(k - 1) * config.tau + stage] * move[l] for k in 1:config.nCarriers for l in config.CarrierIdx[k])
        sum(sum(config.transport_coef[(k - 1) * config.tau + stage] * move[config.CarrierIdx[k]]) for k in 1:config.nCarriers)
    )
end

transportation_t (generic function with 1 method)

In [10]:
# Hyperparameter configuration
config = HyperParams(); # Default

In [11]:
# # Hyperparameter configuration
# config = HyperParams(
#     nOrigins        = 3,
#     Bids            = [[1, 2, 3, 4],
#                        [1, 5, 6],
#                        [2, 3, 4],
#                        [2, 4, 6],
#                        [3, 5]],
#     entry_stock_0   = [5, 5, 5],
#     entry_capacity  = [50, 50, 50],
#     entry_store_coef= [2.0, 2.0, 2.0],
# );

In [12]:
model = SDDP.PolicyGraph(
    (sp, stage) -> transportation_t(sp, stage; config = config),
    SDDP.LinearGraph(config.tau);
    sense       = :Min,
    lower_bound = 0.0,
    optimizer   = HiGHS.Optimizer,
)

A policy graph with 4 nodes.
 Node indices: 1, 2, 3, 4


In [13]:
SDDP.train(model;
    iteration_limit = 20,
    cut_type        = SDDP.SINGLE_CUT,
    parallel_scheme = SDDP.Serial(),
)

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-24
-------------------------------------------------------------------
problem
  nodes           : 4
  state variables : 6
  scenarios       : 4.30467e+07
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [29, 29]
  AffExpr in MOI.EqualTo{Float64}         : [4, 4]
  AffExpr in MOI.LessThan{Float64}        : [7, 7]
  VariableRef in MOI.GreaterThan{Float64} : [19, 19]
  VariableRef in MOI.LessThan{Float64}    : [1, 1]
numerical stability report
  matrix range     [1e+00, 1e+00]
  objective range  [1e+00, 4e+00]
  bounds range     [0e+00, 0e+00]
  rhs range        [5e+00, 5e+01]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solv

## Parallelization of the Multi-Dimensional Instance

In [ ]:
using Distributed
Distributed.addprocs(10);
@everywhere using JuMP, SDDP, Gurobi, DataStructures

In [ ]:
# In Julia this struct is immutable
@everywhere struct HyperParams
    tau             ::Int64
    nOrigins        ::Int64
    nDestinations   ::Int64
    nLanes          ::Int64
    nCarriers       ::Int64
    Bids            ::Vector{Vector{Int64}}
    Winners         ::OrderedDict{Int64, Vector{Int64}}
    CarrierIdx      ::Vector{Vector{Int64}}
    from_           ::Vector{Vector{Int64}}
    to_             ::Vector{Vector{Int64}}
    entry_stock_0   ::Vector{Int64}
    exit_stock_0    ::Vector{Int64}
    exit_short_0    ::Vector{Int64}
    entry_capacity  ::Vector{Int64}
    exit_capacity   ::Vector{Int64}
    flow_support    ::Vector{NTuple}
    entry_store_coef::Vector{Float64}
    exit_store_coef ::Vector{Float64}
    exit_short_coef ::Vector{Float64}
    transport_coef  ::Vector{Float64}
    carrier_capacity::Vector{Int64}
end

# Outer constructor with arguments
@everywhere function HyperParams(; 
                     tau             ::Int64 = 4,
                     nOrigins        ::Int64 = 2,
                     nDestinations   ::Int64 = 2,
                     nCarriers       ::Int64 = 3,
                     Bids            ::Vector{Vector{Int64}} = [[1, 2, 3, 4],
                                                                [1, 3],
                                                                [2, 4],
                                                                [1, 2],
                                                                [3, 4]],
                     Winners         ::OrderedDict{Int64, Vector{Int64}} = OrderedDict(
                                                                             1 => [2, 3], 
                                                                             2 => [4, 5], 
                                                                             3 => [1]),
                     entry_stock_0   ::Vector{Int64} = [5, 5],
                     exit_stock_0    ::Vector{Int64} = [0, 0],
                     exit_short_0    ::Vector{Int64} = [0, 0],
                     entry_capacity  ::Vector{Int64} = [50, 50],
                     exit_capacity   ::Vector{Int64} = [50, 50],
                     flow_support    ::Vector{Int64} = collect(1:10),
                     entry_store_coef::Vector{Float64} = [2.0, 2.0],
                     exit_store_coef ::Vector{Float64} = [1.0, 1.0],
                     exit_short_coef ::Vector{Float64} = [3.0, 3.0],
                     transport_coef  ::Vector{Float64} = [ 2.0, 1.7, 2.1, 3.3,   # Carrier 1
                                                           1.7, 1.9, 2.6, 3.7,   # Carrier 2
                                                           2.4, 1.2, 2.9, 3.1 ], # Carrier 3
                     carrier_capacity::Vector{Int64} = [ 8,  5,  7, 20, 
                                                         7,  5,  8, 20, 
                                                        12,  8,  8, 11])

    # nLanes = nOrigins * nDestinations

    from_ = Dict{Int64, Vector{Int64}}()
    for i in 1:nOrigins
        from_[i] = (collect(1:nDestinations) .- 1) .* nOrigins .+ i
    end

    to_ = Dict{Int64, Vector{Int64}}()
    for j in 1:nDestinations
        to_[j] = (j .- 1) .* nOrigins .+ collect(1:nOrigins)
    end

    ordx = vcat(collect(values(Winners))...)
    Ldx  = vcat(Bids[ordx]...)

    nLc = [0; cumsum([ length(vcat(Bids[Winners[cdx]]...)) for cdx in 1:nCarriers ])]

    HyperParams(
        tau,
        nOrigins,
        nDestinations,
        nLc[end], # nLanes
        nCarriers,
        Bids,
        Winners,
        [ collect(nLc[i] + 1 : nLc[i + 1]) for i in 1:length(nLc) - 1 ], # CarrierIdx
        [ findall(lane -> lane in from_[i], Ldx) for i in 1:nOrigins ],
        [ findall(lane -> lane in to_[j], Ldx) for j in 1:nDestinations ],
        entry_stock_0,
        exit_stock_0,
        exit_short_0,
        entry_capacity,
        exit_capacity,
        vec(collect(Base.product(ntuple(_ -> flow_support, nOrigins + nDestinations)...))),
        entry_store_coef,
        exit_store_coef,
        exit_short_coef,
        transport_coef,
        carrier_capacity)
end

In [ ]:
@everywhere function transportation_t(sp::Model, stage::Int64; config::HyperParams)
    ## State variables
    @variables(sp, begin
        0 <= entry[i = 1:config.nOrigins],      (SDDP.State, initial_value = config.entry_stock_0[i])
        0 <= exitp[j = 1:config.nDestinations], (SDDP.State, initial_value = config.exit_stock_0[j])
        0 <= exitm[j = 1:config.nDestinations], (SDDP.State, initial_value = config.exit_short_0[j])
        0 <= move[k  = 1:config.nLanes]
        inflow[ i = 1:config.nOrigins]
        outflow[j = 1:config.nDestinations]
    end)
    ## Decision variables
    ## Constraints
    # Carrier capacity
    @constraint(sp, [k = 1:config.nCarriers], sum(move[config.CarrierIdx[k]]) <= config.carrier_capacity[(k - 1) * config.tau + stage])
    # # Entry volume
    # @constraint(sp, [i = 1:config.nOrigins], sum(move[config.from_[i]]) <= entry[i].in + inflow[i])
    # Storage limit
    @constraint(sp, [i = 1:config.nOrigins],      entry[i].in + inflow[i] <= config.entry_capacity[i])
    @constraint(sp, [j = 1:config.nDestinations], exitp[j].in + sum(move[config.to_[j]]) <= config.exit_capacity[j])
    # Transition
    @constraint(sp, [i = 1:config.nOrigins],      entry[i].out == entry[i].in + inflow[i] - sum(move[config.from_[i]]))
    @constraint(sp, [j = 1:config.nDestinations], exitp[j].out - exitm[j].out == exitp[j].in - exitm[j].in + sum(move[config.to_[j]]) - outflow[j])
    # Uncertain variables
    Ξ = config.flow_support
    SDDP.parameterize(sp, Ξ) do ξ
        JuMP.fix.(inflow,  ξ[1:config.nOrigins])
        JuMP.fix.(outflow, ξ[config.nOrigins .+ (1:config.nDestinations)])
    end
    ## Objective function
    @stageobjective(sp, 
        sum(config.entry_store_coef[i] * entry[i].in for i in 1:config.nOrigins)      + 
        sum(config.exit_store_coef[j]  * exitp[j].in for j in 1:config.nDestinations) + 
        sum(config.exit_short_coef[j]  * exitm[j].in for j in 1:config.nDestinations) + 
        # sum(config.transport_coef[(k - 1) * config.tau + stage] * move[l] for k in 1:config.nCarriers for l in config.CarrierIdx[k])
        sum(sum(config.transport_coef[(k - 1) * config.tau + stage] * move[config.CarrierIdx[k]]) for k in 1:config.nCarriers)
    )
end

In [ ]:
# Hyperparameter configuration
@everywhere config = HyperParams(); # Default

In [ ]:
model = SDDP.PolicyGraph(
    (sp, stage) -> transportation_t(sp, stage; config = config),
    SDDP.LinearGraph(config.tau);
    sense       = :Min,
    lower_bound = 0.0,
    optimizer   = Gurobi.Optimizer,
)

In [ ]:
SDDP.train(
    model;
    iteration_limit = 10,
    cut_type        = SDDP.SINGLE_CUT,
    parallel_scheme = SDDP.Asynchronous() do m::SDDP.PolicyGraph
        env = Gurobi.Env()
        set_optimizer(m, () -> Gurobi.Optimizer(env))
    end,
)

In [ ]:
# function transportation_t(sp::Model, stage::Int64; config::HyperParams)
#     ## State variables
#     @variables(sp, begin
#         0 <= entry, (SDDP.State, initial_value = config.entry_stock_0[1])
#         0 <= exitp, (SDDP.State, initial_value = config.exit_stock_0[1])
#         0 <= exitm, (SDDP.State, initial_value = config.exit_short_0[1])
#         0 <= move[l = 1:config.nLanes]
#         inflow
#         outflow
#     end)
#     ## Decision variables
#     ## Constraints
#     # Carrier capacity
#     @constraint(sp, sum(move) <= entry.in + inflow)
#     # Storage limit
#     @constraint(sp, entry.in + inflow    <= config.entry_capacity[1])
#     @constraint(sp, exitp.in + sum(move) <= config.exit_capacity[1])
#     # Transition
#     @constraint(sp, entry.out == entry.in + inflow - sum(move))
#     @constraint(sp, exitp.out - exitm.out == exitp.in - exitm.in + sum(move) - outflow)
#     # Uncertain variables
#     Ξ = vec(collect(Base.product(ntuple(_ -> config.flow_support, config.nOrigins + config.nDestinations)...)))
#     SDDP.parameterize(sp, Ξ) do ξ
#         JuMP.fix.(inflow,  ξ[1])
#         JuMP.fix.(outflow, ξ[2])
#     end
#     ## Objective function
#     @stageobjective(sp, 
#         config.entry_store_coef[1] * entry.in + 
#         config.exit_store_coef[1]  * exitp.in + 
#         config.exit_short_coef[1]  * exitm.in + 
#         sum(config.transport_coef[l] * move[l] for l in 1:config.nLanes)
#     )
# end

In [ ]:
# # Hyperparameter configuration
# config = HyperParams(tau             = 4,
#                      nOrigins        = 1,
#                      nDestinations   = 1,
#                      nCarriers       = 2,
#                      Bids            = [[1],
#                                         [1]],
#                      Winners         = OrderedDict(
#                                          1 => [1], 
#                                          2 => [2]),
#                      entry_stock_0   = [5],
#                      exit_stock_0    = [0],
#                      exit_short_0    = [0],
#                      entry_capacity  = [50],
#                      exit_capacity   = [50],
#                      flow_support    = collect(1:10),
#                      entry_store_coef= [2.0],
#                      exit_store_coef = [1.0],
#                      exit_short_coef = [3.0],
#                      transport_coef  = [ 1.1, 0.7 ])

# Recovering `i` and `j` from a 1-Indexed Linear Index in Julia

To recover the row and column indices \( i \) and \( j \) from a 1-indexed linear index \( \ell \) when given \( \ell = (i - 1) \times m + j \) in Julia, you can use the following formulas:

1. Compute $i$ by dividing and flooring: $ i = \left\lfloor \dfrac{\ell - 1}{m} \right\rfloor + 1 $

2. Compute $j$ using the remainder: $ j = (\ell - 1) \bmod m + 1 $


In [ ]:
function get_2d_index(ell, m)
    i = div(ell - 1, m) + 1
    j = (ell - 1) % m + 1
    return i, j
end

In [ ]:
m = 4
i, j = get_2d_index(13, m)
println("i: $i, j: $j")

In [ ]:
using Random

n = 2  # Number of rows
m = 2  # Number of columns
K = 2  # Number of suppliers

# Creating a dictionary to store the 1D indices for each k
kdx = Dict{Int, Vector{Int}}()

# Initialize the dictionary with empty arrays for each k
for k in 1:K
    kdx[k] = Int[]
end

# Loop through each (i, j) and assign indices to suppliers
for i in 1:n
    for j in 1:m
        # Convert (i, j) to a 1D index
        ell = (i - 1) * m + j

        for k in 1:K
            if rand() > 0.5 || ((n <= 2) && (m <= 2))
                # Add the 1D index to the corresponding supplier's list in the dictionary
                push!(kdx[k], ell)
            end
        end
    end
end

# Print the dictionary
for (k, indices) in kdx
    println("Supplier ", k, ": ", indices)
end

In [ ]:
n = 3
m = 2

# Generate 1D indices for row i
i = 1
idx = (collect(1:m) .- 1) .* n .+ i
println("Indices for row \$i: ", idx)

# Generate 1D indices for column j
j = 2
jdx = (j .- 1) .* n .+ collect(1:n)
println("Indices for column \$j: ", jdx)

In [ ]:
intersect(kdx[2], idx)

In [ ]:
# function transportation_t(sp::Model, stage::Int64; n::Int64, m::Int64)
#     @variable(sp, x[i = 1:(n + m)],
#         base_name   = "x",
#         lower_bound = i <= n ? 0 : -10,
#         upper_bound = 10,
#         SDDP.State,
#         initial_value = 0
#     )
# end